**Load the libraries**

In [1]:
from pyprojroot import here
from llama_index.llms import AzureOpenAI
from llama_index.embeddings import AzureOpenAIEmbedding
from llama_index import (set_global_service_context,
                         ServiceContext,
                         SimpleDirectoryReader,
                         VectorStoreIndex,
                         Document,
                         StorageContext,
                         load_index_from_storage)
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

**Load the GPT model and the embedding model from AzureOpenAI**

In [2]:
llm = AzureOpenAI(
    model="gpt-35-turbo-16k",
    engine="gpt-35-turbo-16k",
    deployment_name=os.getenv("gpt_deployment_name"),
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_version=os.getenv("OPENAI_API_VERSION"),
)
embed_model = AzureOpenAIEmbedding(
    model="text-embedding-ada-002",
    deployment_name=os.getenv("embed_deployment_name"),
    api_key=os.getenv("OPENAI_API_KEY"),
    azure_endpoint=os.getenv("OPENAI_API_BASE"),
    api_version=os.getenv("OPENAI_API_VERSION"),
)
# NOTE: Uncomment if you want to use an open source embedding model
# embed_model = "local:BAAI/bge-small-en-v1.5"

**Set the serivce context**

In [3]:
service_context = ServiceContext.from_defaults(
    llm=llm,
    embed_model=embed_model,
)

set_global_service_context(service_context)

**Load the documents**

In [4]:
documents = SimpleDirectoryReader(
    input_files=[here(f"data/docs/{d}") for d in os.listdir(here("data/docs"))]
).load_data()

**Process documents and prepare the index:**

Processing for `Basic RAG`: Turn all the documents in one long piece of text

In [5]:
save_dir=here("data/indexes/basic_index")

In [7]:
document = Document(text="\n\n".join([doc.text for doc in documents]))


index = VectorStoreIndex.from_documents([document],
                                        service_context=service_context)

**Save the index if you wish**

In [ ]:
index.storage_context.persist(save_dir)

**Load the index**

In [ ]:
# Rebuild storage context
storage_context = StorageContext.from_defaults(persist_dir=save_dir)

# Load index from the storage context
index = load_index_from_storage(storage_context)

**Build the query engine**

In [ ]:
query_engine = index.as_query_engine()

**Test with a query**

In [6]:
response = query_engine.query(
    "Explain is the architecture of vision transformer model"
)
print(str(response))

Fred is a small red fish who lived in a cozy coral reef with his loving family. He had a desire for adventure and often explored the reef with his best friend, a playful dolphin named Delphi. One day, they discovered a map that pointed to a hidden treasure located in a distant part of the ocean. Excited by the promise of adventure, Fred kissed his family goodbye and set off on a journey filled with wonders and dangers. Along the way, they encountered towering forests of kelp, evaded a hungry octopus, and met wise old turtles. After enduring a storm and discovering a glowing garden of bioluminescent jellyfish, Fred and Delphi finally reached the sunken pirate ship where the treasure was hidden. They found the treasure chest, but to Fred, the real treasure was the journey itself and the memories he had created with his friend. Triumphant, they returned home to share their adventure with Fred's family.
